# Simple LSTM

In [ ]:
##########################################################################################################
# SOME REFERENCES: https://medium.com/towards-data-science/whats-happening-in-my-lstm-layer-dd8110ecc52f #
##########################################################################################################

#################################################################################
# SOME REFERENCES: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html #
#################################################################################

import torch
import torch.nn as nn

""" Class for a simple LSTM. """
class MyLSTM(nn.Module):
    """ Initialize configurations. """
    def __init__(self, input_size, hidden_size, num_layers, output_size, device, bidirectional=False):
        super(MyLSTM, self).__init__()
        # number of features of x
        self.input_size = input_size
        # number of features in the hidden state h
        self.hidden_size = hidden_size
        # number of recurrent layers
        self.num_layers = num_layers
        # number of output neurons of linear layer
        self.output_size = output_size
        # D parameter
        self.directions = 2 if bidirectional else 1
        # device
        self.device = device

        # lstm-architecture:
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True).to(device)

        # linear-layer
        self.fc1 = nn.Sequential(
            # we take the last cell outputs with shape (batch_size, D * hidden_size)
            nn.Linear(self.directions * self.hidden_size, self.output_size),
            # nn.ReLU(inplace=True)
        )

    """ Method used to define the forward pass of the input through the network during the training. """
    def forward(self, x):
        # input shape (batch_size, sequence_length, number_features) when batch_first=True
        batch_size = x.size(0)
        # (D ∗ num_layers, batch_size, hidden_size)
        h_0 = torch.zeros((self.directions * self.num_layers,
                           batch_size, self.hidden_size)).to(self.device)
        c_0 = torch.zeros((self.directions * self.num_layers,
                           batch_size, self.hidden_size)).to(self.device)

        # output-shape (batch_size, sequence_lenght, D * hidden_size)
        # h_n-shape (D * num_layers, batch_size, hidden_size)
        # c_n-shape (D * num_layers, batch_size, hidden_size)
        output, (h_n, c_n) = self.lstm(x, (h_0, c_0))

        print("\n", output.shape, h_n.shape, c_n.shape)
        print("\n", output, "\n\n", h_n, "\n\n", c_n)

        # takes the last cell outputs of all batches
        print("\nlast cell outputs shape: \n", output[:, -1, :].shape)
        print("\nlast cell outputs: \n", output[:, -1, :])

        lin_out = self.fc1(output[:, -1, :])

        print(f"\nlin-out-shape: {lin_out}")


""" Runs the simulation. """
if __name__ == "__main__":

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"device: {device}")

    x = torch.randn((2, 6, 6))
    print(f"\nx-shape: {x.shape}")

    num_features = int(x.shape[2])

    model = MyLSTM(input_size=num_features,
                   hidden_size=6,
                   num_layers=1,
                   output_size=1,
                   device=device)

    out = model(x)

##################
# output example #
##################

# device: cpu

# x-shape: torch.Size([2, 6, 6])

# torch.Size([2, 6, 6]) torch.Size([1, 2, 6]) torch.Size([1, 2, 6])

# tensor([[[ 0.0670,  0.0398,  0.2406, -0.2882,  0.0184, -0.1168],
#          [-0.0813,  0.2598,  0.3303, -0.1901,  0.1681, -0.1573],
#          [-0.0013,  0.0993,  0.3154, -0.2576,  0.2350, -0.2473],
#          [ 0.1037,  0.0869,  0.3699, -0.1835,  0.0433, -0.3695],
#          [ 0.1005,  0.2060,  0.3575, -0.1126,  0.0005, -0.3399],
#          [ 0.0111,  0.1685,  0.4348, -0.2917,  0.1025, -0.2308]],

#         [[ 0.2161, -0.0216,  0.1619, -0.0560, -0.1214, -0.2652],
#          [ 0.1342,  0.1155,  0.3858, -0.1652, -0.1072, -0.2497],
#          [-0.0412,  0.2684,  0.1980, -0.1152,  0.2417, -0.0435],
#          [-0.4885,  0.4563, -0.0637, -0.1715,  0.2013,  0.0407],
#          [-0.1746,  0.2125,  0.0317, -0.2900,  0.3504, -0.0428],
#          [-0.3115,  0.3664, -0.0838, -0.2712,  0.2742, -0.0829]]],
#        grad_fn=<TransposeBackward0>)

# tensor([[[ 0.0111,  0.1685,  0.4348, -0.2917,  0.1025, -0.2308],
#          [-0.3115,  0.3664, -0.0838, -0.2712,  0.2742, -0.0829]]],
#        grad_fn=<StackBackward0>)

# tensor([[[ 0.0251,  0.3550,  0.9562, -0.6240,  0.2229, -0.6161],
#          [-0.4754,  0.7592, -0.2685, -0.7482,  0.9308, -0.2028]]],
#        grad_fn=<StackBackward0>)

# last cell outputs shape: torch.Size([2, 6])

# last cell outputs:
# tensor([[ 0.0111,  0.1685,  0.4348, -0.2917,  0.1025, -0.2308],
#         [-0.3115,  0.3664, -0.0838, -0.2712,  0.2742, -0.0829]],
#        grad_fn=<SliceBackward0>)

# lin-out-shape:
# tensor([[0.2285],
#         [0.3122]], grad_fn=<ReluBackward0>)


# Autoencoder LSTM

In [103]:
##########################################################################################################
# SOME REFERENCES: https://medium.com/towards-data-science/whats-happening-in-my-lstm-layer-dd8110ecc52f #
##########################################################################################################

#################################################################################
# SOME REFERENCES: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html #
#################################################################################

import torch
import torch.nn as nn

""" Class form LSTM in Autoencoder configuration. """
class MyAutoEncLSTM(nn.Module):
    """ Initialize configurations. """
    def __init__(self, input_size, hidden_size, embedding_dim, num_layers, device, bidirectional=False):
        super(MyAutoEncLSTM, self).__init__()
        # the number of expected features in the input x
        self.input_size = input_size
        # the number of features in the hidden state h
        self.hidden_size = hidden_size
        # the hidden-size of the last encoder layer
        self.embedding_dim = embedding_dim
        # number of recurrent layers
        self.num_layers = num_layers
        # if true becomes a bidirectional LSTM
        D = 2 if bidirectional else 1
        self.directions = D
        # device where to put tensors
        self.device = device

        # lstm-architecture
        self.lstm_encoder1 = nn.LSTM(input_size=input_size,
                                     hidden_size=hidden_size,
                                     num_layers=num_layers,
                                     batch_first=True)
        # lstm-architecture
        self.lstm_encoder2 = nn.LSTM(input_size=D*hidden_size,
                                     hidden_size=embedding_dim,
                                     num_layers=num_layers,
                                     batch_first=True)
        # # lstm-architecture
        # self.lstm_decoder1 = nn.LSTM(input_size=input_size,
        #                              hidden_size=hidden_size,
        #                              num_layers=num_layers,
        #                              batch_first=True)
        # # lstm-architecture
        # self.lstm_decoder2 = nn.LSTM(input_size=input_size,
        #                              hidden_size=hidden_size,
        #                              num_layers=num_layers,
        #                              batch_first=True)

    """ Method used to define the forward pass of the input through the network during the training. """
    def forward(self, x):
        # input shape (batch_size, sequence_length, number_features) when batch_first=True

        # output-shape (batch_size, sequence_lenght, D * hidden_size)
        # h_n-shape (D * num_layers, batch_size, hidden_size)
        # c_n-shape (D * num_layers, batch_size, hidden_size)
        enc1_output, (_, _) = self.lstm_encoder1(x)

        print(f"\nenc1-output-shape: \n{enc1_output.shape}")
        print(f"\nenc1-output: \n{enc1_output}")

        enc2_output, (hidden_n, _) = self.lstm_encoder2(enc1_output)

        print(f"\nenc2-output-shape: \n{enc2_output.shape}")
        print(f"\nenc2-output: \n{enc2_output}")
        print(f"\nhidden_n-shape: \n{hidden_n.shape}")
        print(f"\nhidden_n: \n{hidden_n}")

        # prendo enc2_output oppure hidden_n ? --> l'architettura di rete cambia


""" Runs the simulation. """
if __name__ == "__main__":
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"\ndevice: \n{device}")

    batch_size = 2
    seq_len = 2  # 5
    num_features = 3  # 18
    hidden_size = 2
    embedding_dim = 1
    num_layers = 1
    x = torch.rand((batch_size, seq_len, num_features))

    print(f"\ninput-shape: \n{x.shape}")
    print(f"\ninput: \n{x}")

    # model definition
    model = MyAutoEncLSTM(input_size=num_features,
                          hidden_size=hidden_size,
                          embedding_dim=embedding_dim,
                          num_layers=num_layers,
                          device=device)

    # model output
    output = model(x)



device: 
cpu

input-shape: 
torch.Size([2, 2, 3])

input: 
tensor([[[0.1134, 0.5940, 0.2335],
         [0.6617, 0.2146, 0.2131]],

        [[0.3208, 0.7989, 0.0239],
         [0.9745, 0.4620, 0.1497]]])

enc1-output-shape: 
torch.Size([2, 2, 2])

enc1-output: 
tensor([[[-0.0830, -0.1466],
         [-0.1376, -0.1690]],

        [[-0.0529, -0.1436],
         [-0.0880, -0.1840]]], grad_fn=<TransposeBackward0>)

enc2-output-shape: 
torch.Size([2, 2, 1])

enc2-output: 
tensor([[[0.0273],
         [0.0558]],

        [[0.0245],
         [0.0473]]], grad_fn=<TransposeBackward0>)

hidden_n-shape: 
torch.Size([1, 2, 1])

hidden_n: 
tensor([[[0.0558],
         [0.0473]]], grad_fn=<StackBackward0>)
